In [1]:
import numpy as np
import qiskit
import qiskit.quantum_info as qi
from qoop.core import state, metric, ansatz
from qoop.compilation.qcompilation import QuantumCompilation


In [32]:

n_test = 10
tests = []

fidelitiess = []

depthss = []

for n in range(2, 7):
    tests = []
    fidelities = []
    depths = []
    for i in range(0, n_test):
        random_state = np.random.uniform(0, 2*np.pi, 2**n)
        random_state = random_state/np.linalg.norm(random_state)
        tests.append(np.array(random_state))

    for i in range(0, n_test):
        qc = qiskit.QuantumCircuit(n)
        qc.prepare_state(tests[i])
        from qiskit.providers.basic_provider import BasicProvider

        backend = BasicProvider().get_backend('basic_simulator')
        qc_basis = qiskit.transpile(qc, backend, 
            basis_gates=['h', 'cx',
                        'rx','ry','rz',
                        'crx','cry','crz'])
        depths.append(qc_basis.depth())
        rho = qi.DensityMatrix(qc)
        tests[i] = np.expand_dims(tests[i], axis=0)
        sigma =  np.transpose(np.conjugate(tests[i])) @ tests[i]
        fidelity = metric.compilation_trace_fidelity(rho, sigma)
        fidelities.append(fidelity)
    print(qc_basis.count_ops())
    fidelitiess.append(np.mean(fidelities))
    depthss.append(np.mean(depths))
print(fidelitiess)
print(depthss)

Failed to find a square root.
Failed to find a square root.
OrderedDict([('ry', 3), ('cx', 2)])
OrderedDict([('ry', 7), ('cx', 6)])
OrderedDict([('ry', 15), ('cx', 14)])
OrderedDict([('ry', 31), ('cx', 30)])
OrderedDict([('ry', 63), ('cx', 62)])
[nan, 1.0000000000000002, 0.9999999999999997, 1.000000000000001, 0.9999999999999524]
[4.0, 11.0, 26.0, 57.0, 120.0]


In [35]:
for n in [2,3,4,5,6]:
    from qiskit.providers.basic_provider import BasicProvider
    qc = ansatz.g2gn(n, 2)
    backend = BasicProvider().get_backend('basic_simulator')
    qc_basis = qiskit.transpile(qc, backend, 
        basis_gates=['h', 'cx',
                    'rx','ry','rz',
                    'crx','cry','crz'])
    print(len(qc.parameters))
    print(qc_basis.count_ops())


12
OrderedDict([('ry', 12), ('h', 12), ('cx', 6)])
18
OrderedDict([('ry', 18), ('cx', 18), ('rz', 14), ('h', 8)])
24
OrderedDict([('rz', 56), ('cx', 52), ('ry', 32), ('h', 16)])
30
OrderedDict([('rz', 102), ('cx', 98), ('ry', 34), ('h', 16), ('rx', 6)])
36
OrderedDict([('rz', 186), ('cx', 180), ('ry', 56), ('h', 24), ('rx', 6)])


In [10]:
def graph(num_qubits, random_state):
    num_layers = 2
    compiler =  QuantumCompilation(
        u = ansatz.g2gn(num_qubits, num_layers),
        vdagger = state.specific(random_state).inverse(),
        metrics_func=[
            'loss_basic',
            'compilation_trace_fidelities'
        ]
    )
    compiler.fast_fit(1)
    return compiler.metrics['compilation_trace_fidelities'][-1]
n_test = 10
def run(n):
    tests = []
    fidelities = []
    depths = []
    for i in range(0, n_test):
        random_state = np.random.uniform(0, 2*np.pi, 2**n)
        random_state = random_state/np.linalg.norm(random_state)
        tests.append(np.array(random_state))

    for i in range(0, n_test):
        fidelities.append(graph(n))
    print(f"{n} qubits: ", np.mean(fidelities))
for n in  [2,3,4,5,6]:
    run(n)

TypeError: graph() missing 1 required positional argument: 'random_state'

In [1]:
import numpy as np
import qiskit
import qiskit.quantum_info as qi
from qoop.core import state, metric, ansatz
from qoop.compilation.qcompilation import QuantumCompilation
from qoop.backend import utilities

def graph(qc, random_state):
    compiler =  QuantumCompilation(
        u = qc,
        vdagger = state.specific(random_state).inverse(),
        metrics_func=[
            'loss_basic',
            'compilation_trace_fidelities'
        ]
    )
    compiler.fast_fit(100) 
    return compiler.metrics['compilation_trace_fidelities'][-1]

n_test = 10


def bypass_compile(num_qubits):
    tests = []
    
    depths = []
    if num_qubits == 2:
        depth = [3]
    elif num_qubits == 3:
        depth = range(5,15)
    elif num_qubits == 4:
        depth = range(5, 30)
    elif num_qubits == 5:
        depth = range(5, 40)
    elif num_qubits == 6:
        depth = range(5, 40)
    index = 0
    for i in range(0, n_test):
        random_state = np.random.uniform(0, 2*np.pi, 2**n)
        random_state = random_state/np.linalg.norm(random_state)
        tests.append(np.array(random_state))
    while(True or index < len(depth)):
        fidelities = []
        qc = utilities.load_circuit(f'data/n={num_qubits},d={depth[index]},n_circuit=32,n_gen=50/best_circuit')
        for i in range(0, n_test):
            fidelities.append(graph(qc, tests[i]))
        print(f"n={num_qubits}, d={depth[index]}, f={np.mean(fidelities)}")
        if np.mean(fidelities) < 0.99:
            index += 1
            if index == len(depth):
                break
        else:
            break
for n in [2,3,4,5,6]:
    bypass_compile(n)

[]


ValueError: shapes (0,) and (4,) not aligned: 0 (dim 0) != 4 (dim 0)

In [11]:
  
# if __name__ == '__main__':
#     ns = [2,3,4,5,6]
#     multiple_compile(ns)
# n=5, d=3, f=0.7938522787939981
# n=5, d=4, f=0.5094050486111035
import numpy as np
import qiskit
import qiskit.quantum_info as qi
from qoop.core import state, metric, ansatz
from qoop.compilation.qcompilation import QuantumCompilation
from qoop.backend import utilities


n_test = 10

def multiple_compile(params):
    import concurrent.futures
    executor = concurrent.futures.ProcessPoolExecutor()
    results = executor.map(bypass_compile, params)
    return results

def graph(qc, random_state):
    compiler =  QuantumCompilation(
        u = qc,
        vdagger = state.specific(random_state).inverse(),
        metrics_func=[
            'loss_basic',
            'compilation_trace_fidelities'
        ]
    )
    compiler.fast_fit(100) 
    return compiler.metrics['compilation_trace_fidelities'][-1]

def bypass_compile(num_qubits):
    tests = []
    if num_qubits == 2:
        depth = [6,7,8,9]
    elif num_qubits == 3:
        depth = [2,4]
    elif num_qubits == 4:
        depth = range(3, 5)
    elif num_qubits == 5:
        depth = range(4, 5)
    elif num_qubits == 6:
        depth = range(25, 40)
    index = 0
    for i in range(0, n_test):
        random_state = np.random.uniform(0, 2*np.pi, 2**num_qubits)
        random_state = random_state/np.linalg.norm(random_state)
        tests.append(np.array(random_state))
    while(True or index < len(depth)):
        fidelities = []
        qc = utilities.load_circuit(f'data/n={num_qubits},d={depth[index]},n_circuit=32,n_gen=50/best_circuit')
        for i in range(0, n_test):
            #print(tests[i])
            fidelities.append(graph(qc, tests[i]))
        print(f"n={num_qubits}, d={depth[index]}, f={np.mean(fidelities)}")
        index += 1
bypass_compile(3)       
# if __name__ == '__main__':
#     ns = [2,3,4,5,6]
#     multiple_compile(ns)
# n=5, d=3, f=0.7938522787939981
# n=5, d=4, f=0.5094050486111035
# n=4, d=3, f=0.8578980378037141
# n=4, d=4, f=0.8232849000754479
# n=3, d=2, f=0.8677446885901313
# n=3, d=4, f=0.8525561406821124

n=3, d=2, f=0.8677446885901313
n=3, d=4, f=0.8525561406821124


IndexError: list index out of range

In [ ]:
f2 = [0.5773293822752275, 0.9402817802836569]
f3 = [9401234389504806, 0.9567633634775301, 
      0.963731607779609, 0.9878707619498597, 
      0.9972726119280582, 0.9985858406138128,
      0.9989260643216443, 0.9985424196132385,
      0.9999373169820546, 0.9998850020786249]

f4 = []
f5 = []